# Matrices for data scientists

#### Guillermo Moncecchi

If you wanto to do machine learning, you need to work with matrices with the same fluency you have with numbers.This notebooks aims to review the main properties of matrices, and inspect how to manipulate them using  `numpy` and `scipy` (see [the difference](http://www.scipy.org/scipylib/faq.html#what-is-the-difference-between-numpy-and-scipy)). It is not a numpy/scipy tutorial, but a brief introduction to matrices, showing also how to manipulate them using Python.


In [1]:
import numpy as np, scipy as sp

### 1. Very basic operations

First, let's create some numpy arrays, the common structure for dealing with n-dimensional matrices. I will mostly work with two-dimensional matrices since are easier to interpret, but most results admints n-dimensional  matrices (I guess...). The reference I used here were the first chapters of the [notes](http://imerl.fing.edu.uy/gal1/Curso_2011/main.pdf) (pdf) for the "Geometría y Álgebra Lineal" from Universidad de la República, Uruguay, a course I took many many years ago. For a  tutorial on `numpy`, refer to [this](http://www.engr.ucsb.edu/~shell/che210d/numpy.pdf) one, by M. Scott Shell. 

First, let's create a 2D-numpy array, and show that its elements are integers.

In [2]:
# Specify each row of the matrix as a Python list
a=np.array([[0,0,1],[1,1,0],[1,1,1]])
print a
print a.dtype

[[0 0 1]
 [1 1 0]
 [1 1 1]]
int64


Create an matrix of 3 rows by 4 columns, filled with zeros and another filled with ones.

In [3]:
print np.zeros((3,4))


[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


In [4]:
print np.ones((3,4))

[[ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]]


1D-arrays in Python are always row vectors, i.e. vectors are just $1\times n$ matrices:

In [64]:
print np.ones(10)
print np.ones(10).size

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
10


First operation: *matrix sum* (both matrices should have the same dimensionns)

In [5]:
a=np.array([0,1,1,0,0,1,1,1,1]).reshape((3,3))
b=np.array([1,1,1,1,0,1,0,1,1]).reshape((3,3))
print a + b


[[1 2 2]
 [1 0 2]
 [1 2 2]]


If the two matrixs have different dimensions, numpy _upcasts_ matrixes (adding elements to make both matrixes compatibles). In the following example the only row in b is repeated to make b a 2x2 matrix

In [6]:
a=np.array([0,1,2,1]).reshape((2,2))
b=np.array([0,1])

print a
print b
print a + b

[[0 1]
 [2 1]]
[0 1]
[[0 2]
 [2 2]]


Second operation: multiply a scalar by a matrix. 

In [7]:
a=8.1
b=np.ones((2,3))
print a*b

[[ 8.1  8.1  8.1]
 [ 8.1  8.1  8.1]]


Third operation: Matrix product. Given $A(n,p)$ and $B(p,m)$, $A.B=C$, where $C=((c_{ij}))$ and $c_{ij}=\Sigma_{h=1}^p a_{ih}b_{hj}$

In [8]:
A=np.array([1,2,1,0,1,1,0,1,1,0,0,0]).reshape(4,3)
print A
B=np.array([[2,1],[1,1],[0,0]])
print B

[[1 2 1]
 [0 1 1]
 [0 1 1]
 [0 0 0]]
[[2 1]
 [1 1]
 [0 0]]


In [9]:
print A.dot(B) 

[[4 3]
 [1 1]
 [1 1]
 [0 0]]


or...

In [10]:
print np.dot(A,B)

[[4 3]
 [1 1]
 [1 1]
 [0 0]]


I find easier to see matrix product as a matrix where each row in the product is the dot product of the corresponding row in A ($A_i$) and B. numpy allows to slice columns and rows:

In [11]:
print "Shape of A:",A.shape
for i in range(A.shape[0]):
    print np.dot(A[i],B)

Shape of A: (4, 3)
[4 3]
[1 1]
[1 1]
[0 0]


Alternatively, we can see the product matrix as a matrix where each column is the dot product of A and the corresponding column in B ($B^j$)

In [12]:
for j in range(B.shape[1]):
    print np.dot(A,B[:,j])

[4 1 1 0]
[3 1 1 0]


Note that resulting arrays are row arrays, not (as you could have expected) column arrays. The reason is simple: 1-D arrays in numpy are all the same (no matter if they row or column).

*Matrix transposition*: change columns into rows, and viceversa

In [13]:
print A
print np.transpose(A)

[[1 2 1]
 [0 1 1]
 [0 1 1]
 [0 0 0]]
[[1 0 0 0]
 [2 1 1 0]
 [1 1 1 0]]


An interesting property (prove it!): 


$(A\cdot B)^t = B^t \cdot A^t$ **[1]**


In [14]:
print np.transpose(A.dot(B))
print np.transpose(B).dot(np.transpose(A))

[[4 1 1 0]
 [3 1 1 0]]
[[4 1 1 0]
 [3 1 1 0]]


### 2. Matrices as linear transformations

From [Wikipedia](http://en.wikipedia.org/wiki/Matrix_%28mathematics%29): 

_A major application of matrices is to represent linear transformations, that is, generalizations of linear functions such as f(x) = 4x. For example, the rotation of vectors in three dimensional space is a linear transformation which can be represented by a rotation matrix R. If v is a column vector (a matrix with only one column) describing the position of a point in space, the product Rv is a column vector describing the position of that point after a rotation. The product of two matrices is a matrix that represents the composition of two linear transformations.

The following picture shows how the unit square is transformed under a linear transformation represented by a 2-by-2 matrix. I found that this use of matrices can let us imagine better the abstract properties we will describe. 



In [15]:
from IPython.display import Image
Image(url='http://upload.wikimedia.org/wikipedia/commons/a/ad/Area_parallellogram_as_determinant.svg',width=350)


The vectors represented by a 2-by-2 matrix correspond to the sides of a unit square transformed into a parallelogram.
Let's show this in Python. Suppose we have the matrix [[1,1],[2,1]]. This matrix represents a rotation, where each of point in $R^2$ (x,y) is mapped to (x+2y,x+y). Let's verify this:

In [16]:
Ex=np.array([[1,1],[2,1]])
print Ex

print np.dot([1,1],Ex) # [1+2,1+1]

[[1 1]
 [2 1]]
[3 2]


Let's see how the unit square is transformed

In [17]:
us=np.array([[0,0],[0,1],[1,0],[1,1]])
print np.dot(us,Ex)

[[0 0]
 [2 1]
 [1 1]
 [3 2]]


### 3. Inverse Matrix

In [18]:
import numpy.linalg

An $(n,n)$ matrix A is _invertible_ if there is a matrix B such that $A \cdot B = I$, where $I$ is the identity matrix. B is the _inverse matrix_ of $A$. 

In [19]:
A=np.array([1,1,1,0,1,1,1,2,1]).reshape((3,3))
B=np.linalg.inv(A)
print B

[[ 1. -1.  0.]
 [-1.  0.  1.]
 [ 1.  1. -1.]]


In [20]:
print np.dot(A,B)

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]


In [21]:
print np.dot(B,A)

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]


Find the inverse of some matrix:

In [22]:
A=np.array([2,-2,-1,3]).reshape((2,2))
print np.linalg.inv(A)

[[ 0.75  0.5 ]
 [ 0.25  0.5 ]]


For linear transformations (you guessed it), inverse matrices represent the linear transformation that gives back the original plane.

### 4.Basic Matrix Identities (from Bishop's book)

The notes for this section (and for the following ones) try to create an numpy version of the notes in appendix C of the _"Pattern Recognition and Machine Learning"_ book by Bishop, called "Properties of Matrices"

In [23]:
A=np.array([1,1,1,0,1,1,1,2,1]).reshape((3,3))
B=np.array([1,0,1,0,1,1,0,1,2]).reshape((3,3))
print A
print B


[[1 1 1]
 [0 1 1]
 [1 2 1]]
[[1 0 1]
 [0 1 1]
 [0 1 2]]


*Property 1:*  

$A\cdot A^{-1} = A^{-1}\cdot A = I$ **[2]**.

That is, _if we multiply a matrix by its inverse, we get the identity matrix (no matter the product order)_

In [24]:
print np.dot(A,np.linalg.inv(A))

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]


In [25]:
print np.dot(np.linalg.inv(A),A)

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]


_Property 2_: $(AB)^{-1} = B^{-1}A^{-1}$ **[3]**. Proof: just left multiply with $AB$ both sides and get $I$

In [26]:
print np.linalg.inv(np.dot(A,B))
print  np.dot(np.linalg.inv(B),np.linalg.inv(A))

[[-1. -2.  2.]
 [-3. -1.  3.]
 [ 2.  1. -2.]]
[[-1. -2.  2.]
 [-3. -1.  3.]
 [ 2.  1. -2.]]


_Property 3_: $(A^T)^{-1}= (A^{-1})^T$ **[4]**. _The inverse of the trasposed is the trasposed inverted_ 

Proof: using Property 1 and transposing, we get $(AA^{-1})^{T}=I$. Applying [1], we get $(A^{-1})^T A^T=I$. Multiplying both sides to the right by  $(A^T)^{-1}$, we get the result we were searching for. Let's verify it using our examples:

In [27]:
print np.linalg.inv(np.transpose(A))
print np.transpose(np.linalg.inv(A))

[[ 1. -1.  1.]
 [-1.  0.  1.]
 [-0.  1. -1.]]
[[ 1. -1.  1.]
 [-1.  0.  1.]
 [ 0.  1. -1.]]


Some definitions. A set of vectors is _linearly independent_ if none of the vectors can be expressed as a lineal combination of the remainder (or, equivalently, if $\sum_{n} \alpha_{n}a_n = 0$ only if all $\alpha_{n}$). The _rank_ of a matrix is the maximum number of linearly independent rows (or, equivalently, columns)

### 4. Square Matrixs. Traces and determinants.

The _trace_ of a matrix is defined as the sum of the elements of its diagonal. 

In [28]:
print A
print np.trace(A)

[[1 1 1]
 [0 1 1]
 [1 2 1]]
3


We have $Tr(AB)=Tr(BA)$ **[5]**

In [29]:
print np.trace(np.dot(A,B)),"=",np.trace(np.dot(B,A))

8 = 8


The determinant $|A|$ of a $N\times N$ matrix is defined by:

$|A| = \sum (\pm 1)A_{1i_1}A_{2i_2}\ldots A_{Ni_N}$, where "the sum is taken over all products consisting of precisely one element from each row and one element from each column, with a coefficient $+1$ or $-1$ according to whether the permutation $i_1i_2\ldots i_N$ is even or odd, respectively".  From Wikipedia: _"A geometric interpretation can be given to the value of the determinant of a square matrix with real entries: the absolute value of the determinant gives the scale factor by which area or volume (or a higher-dimensional analogue) is multiplied under the associated linear transformation, while its sign indicates whether the transformation preserves orientation. Thus a 2 × 2 matrix with determinant −2, when applied to a region of the plane with finite area, will transform that region into one with twice the area, while reversing its orientation"_



Property: $|AB| = |A||B|$ **[6]** (The determinant of a product of matrices is the product of their determinants)

In [30]:
print np.linalg.det(np.dot(A,B))
print np.linalg.det(A), np.linalg.det(B)

-1.0
-1.0 1.0


Determinant of the inverse matrix: $|A^{-1}| = \frac{1}{|A|}$  **[7]**(The determinant of the inverse of a matrix is the real inverse of its determinant). Note that this implies that a matrix is invertible if it has a non-zero determinant!

In [31]:
C=np.array([2,-2,-1,3]).reshape((2,2))
print np.linalg.det(np.linalg.inv(C))
print np.linalg.det(C)

0.25
4.0


If A and B are of size $N \times M$, then $|I_N + AB^T| = |I_M + A^TB|$ **[7]** A special case is: $|I_N + ab^T| = |1 + a^Tb|$, where a and b are N-dimensional column vectors

In [32]:
a=np.array([1,2,3,4]).reshape(4,1)
b=np.array([1,2,1,2]).reshape(4,1)
print np.eye(4) + np.dot(a,np.transpose(b))
print np.linalg.det(np.eye(4)+np.dot(a,np.transpose(b)))
print 1.0 + np.dot(np.transpose(a).reshape(4),b.reshape(4))

[[ 2.  2.  1.  2.]
 [ 2.  5.  2.  4.]
 [ 3.  6.  4.  6.]
 [ 4.  8.  4.  9.]]
17.0
17.0


### 5. Eigenvector Equation

If A is of size $M \times M$, then the eigenvector equation is $Au_i = \lambda _i u_i$ ($u_i$ is an _eigenvector_ and $\lambda_i$ is its corresponding _eigenvalue_). If you have a vector $v$, and a matrix $A$, the dot product $Av$ produces another vector $v'$. If $v$ and $v'$ are parallel, then $v$ is an eigenvector of $A$, and the scale factor between $v$ and $v'$ is its eigenvalue. In the following picture (taken from the beautiful Wikipedia page for Eigenvalues and Eigenvectors), the transformation produced by the matrix [[2,1],[1,2]] is shown; blue and pink vectors are eigenvectors:

In [33]:
from IPython.display import Image
Image(url='http://upload.wikimedia.org/wikipedia/commons/a/ad/Eigenvectors-extended.gif')

The eigenvalue equation for $A$ is $(A -\lambda I)v =0$. This equation has a non-zero solution iif its determinant $|A - \lambda I|=0$; its roots are precisely the eigenvalues of A.

Suppose we have the following A (Wikipedia, again!):

In [34]:
A=np.array([2,0,0,0,3,4,0,4,9]).reshape(3,3)
print A

[[2 0 0]
 [0 3 4]
 [0 4 9]]


Its rank is the number of nonzero eigenvalues:

In [35]:
print np.linalg.matrix_rank(A)

3


Find the eigenvalues and eigenvectors (note that any non-zero multiple of an eigenvector is also an eigenvector)

In [36]:
print np.linalg.eig(A)[0] #Eigenvalues
print np.linalg.eig(A)[1] #Eigenvectors


[ 11.   1.   2.]
[[ 0.          0.          1.        ]
 [ 0.4472136   0.89442719  0.        ]
 [ 0.89442719 -0.4472136   0.        ]]


### 6. Symmetric Matrices

This matrices have the property that $A^T=A$. The inverse of a matrix is also symmetric (demonstrate). For symmetric matrices, the eigenvalues are real. See, for example:

In [37]:
A=np.array([3,1,1,1,5,0,1,0,1]).reshape(3,3)
print A
print np.linalg.eigvals(A)

[[3 1 1]
 [1 5 0]
 [1 0 1]]
[ 0.55051026  3.          5.44948974]


The eigenvectors of a matrix can be choosen to be orthonormal (i.e. $u_i^Tu_j = I_{ij}$, and, by normalizing, can be set to unit length)

In [38]:
ev=np.linalg.eig(A)[1]
print np.dot(ev[0],ev[1])
print np.dot(ev[1],ev[2])
print np.dot(ev[1],ev[1])
print np.linalg.norm(ev[1])

1.66533453694e-16
-1.38777878078e-16
1.0
1.0


Any M-dimensional vector can be expressed as a linear combination of the eigenvectors.

Let's define an $M \times M$ matriz **U**, where the eigenvectors $u_i$ are the columns. This matrix satisfies $U^TU=I$. This matrix is _orthogonal_. The eigenvector equation can be defined now as: $AU = U\Lambda$ **[8]**, where $\Lambda$ is a diagonal matrix whose elements are the eigenvalues. It can be shown that multiplication by $U$ can be interpreted as a rigid rotation of the coordinate system.

In [39]:
# Show the eigenvectors of A
print ev


[[ 0.40824829 -0.81649658 -0.40824829]
 [-0.09175171  0.40824829 -0.90824829]
 [-0.90824829 -0.40824829 -0.09175171]]


In [40]:
# Verify that AU = U\Lambda
print np.dot(A,ev)

lmbda= np.diag(np.linalg.eigvals(A))
print np.dot(ev,lmbda)

[[ 0.22474487 -2.44948974 -2.22474487]
 [-0.05051026  1.22474487 -4.94948974]
 [-0.5        -1.22474487 -0.5       ]]
[[ 0.22474487 -2.44948974 -2.22474487]
 [-0.05051026  1.22474487 -4.94948974]
 [-0.5        -1.22474487 -0.5       ]]


From $AU = U\Lambda$, it follows $U^TAU = \Lambda$  ($A$ is _diagonalized_ by $U$). From this, left multiplying by $U$, right multiplying by $U^T$, we get 

$A = U \Lambda U^T$, or $A = \sum_{i=1}^M {\lambda_i} u_i u_i^T$ **[9]**

Taking inverse and because $U^{-1} = U^T$, we have: 

$A^{-1} = U \Lambda^{-1} U^T$, or $A^{-1} = \sum_{i=1}^M \frac{1}{\lambda_i} u_i u_i^T$ **[10]**

In [41]:
# This should return A
print ev.dot(lmbda.dot(np.transpose(ev)))

[[  3.00000000e+00   1.00000000e+00   1.00000000e+00]
 [  1.00000000e+00   5.00000000e+00  -1.02001740e-15]
 [  1.00000000e+00  -8.88178420e-16   1.00000000e+00]]


In [42]:
# Calculate the inverse of A
print ev.dot(np.linalg.inv(lmbda).dot(np.transpose(ev)))

print np.linalg.inv(A)


[[ 0.55555556 -0.11111111 -0.55555556]
 [-0.11111111  0.22222222  0.11111111]
 [-0.55555556  0.11111111  1.55555556]]
[[ 0.55555556 -0.11111111 -0.55555556]
 [-0.11111111  0.22222222  0.11111111]
 [-0.55555556  0.11111111  1.55555556]]


We can also show that the determinant of a matrix is the products of its eigenvalues:

$|A| = \prod_{i=1}^{M}\lambda_i$ **[11]**

And that the trace of a matrix is the sum of its eigenvalues

$Tr(A) = \sum_{i=1}^{M}\lambda_i$ **[12]**



In [43]:
print np.prod(np.linalg.eigvals(A)), np.linalg.det(A)

print np.sum(np.linalg.eigvals(A)), np.trace(A)

9.0 9.0
9.0 9


A matrix A is _positive (semi)definite_ if, for all w, $w^T A w >(\geq) 0$. A positive definite matrix has all positive eigenvalues.


### 7. Matrix Derivatives

Given a function $f$ mapping from $m$-by-$n$ matrices to real numbers, we can define the _gradient_ $\nabla_A f(A)$ as an $m$-by-$n$ matrix whose $(i,j)$ element is $\partial f/\partial A_{ij}$. Some properties of gradients (see Bishop's appendix for details), are the following:

- $\frac{\partial}{\partial A}Tr(AB)=B^T$ **[11]**
- $\frac{\partial}{\partial A}Tr(A^TB)=B$ **[12]**
- $\frac{\partial}{\partial A}Tr(A)=I$ **[13]**
- $\frac{\partial}{\partial A}Tr(ABA^T)=A(B+B^T)$**[14]**

And the beautiful

- $\frac{\partial}{\partial A}\ln|A|=(A^{-1})^T$**[15]**

Yet another two properties of gradients (from Andrew Ng's course notes, see next section):

- $\frac{\partial}{\partial{A^T}} f(A)= (\frac{\partial}{\partial A} f(A))^T$ **[16]**
- $\frac{\partial}{\partial{A}} Tr(ABA^TC) = CAB + C^TAB^T$ **[17]**

### 8. Matrices for Machine Learning

It is difficult to find a machine learning method that do _not_ use matrices. From linear regression to deep neural networks, going through Principal Component Analysis, things are much easier if we use matrices and vectors. Just as an example, let's show how find a closed-form solution for the problem of linear regression, following exactly the magnificient [notes](http://cs229.stanford.edu/notes/cs229-notes1.pdf) of Mr. @andrewyng. In the next section, we will see how to implement everything using Python.

First, let's define the problem of linear regression: we have a number of input _features_ from $\mathbb{R}^n$, noted as $x$, and an associated _target value_ $y$ in $\mathbb{R}$. Our _hypothesis_ is that, given $x$, $h(x) = \sum_{i=0}^{i=n} \theta_i x_i$ (assuming that $x_0=1$), or, using matrix notation (see how easier it looks):

$h(x)=\theta^T x$



(Note that, by convention, and instance from $\mathbb{R}^n$ is considered a column vector). What we want to do is to find $\theta^T$, given a training set (i.e. instances from $\mathbb{R}^n$ and their associated target value). The cost function we want to minimize wrt $\theta$ is the following:

$J(\theta) = \frac{1}{2}\sum_{i=1}^{i=m} (h_\theta(x^{(i)})- y^{(i)})^2$

where $x^{(i)}$ denotes the i-th instance, and $y^{(i)}$ its value. Note we are squaring the difference to take into account only the absolute value of the differences. 

This method is called _ordinary least squares_.


A typical way for minimizing $J$ is using some numerical method, i.e. _gradient descent_, which allows to minimize any convex function of multiple variables. For ordinary least squares, however, there is a _closed form_ solution, which we will show. The use of matrix notation will make things much easier than if we had to use index notation. 

First, let's rewrite J using matrix notation.

$J(\theta) = \frac{1}{2}(X\theta - y)(X\theta - y)$

where $X$ is the _design matrix_ that includes the training instances input values as rows, and $y$ is a column matrix including the corresponding target values (you can use index notation to check the equivalence, keeping in mind that $zz^T$ equals the sum of the squared elements of $z$!)




From now on, things are easy (especially if you compare to using index notation). First, use [1]:

$$
\nabla_\theta J(\theta) =\nabla_\theta\frac{1}{2}(X\theta - y)^T(X\theta - y) = \nabla_\theta\frac{1}{2} (\theta^TX^TX\theta - \theta^TX^Ty - y^TX\theta+ yy^T) 
$$


Now, since $J(\theta)$ is a real number, its trace is the same number

$$
\nabla_\theta J(\theta) =\nabla_\theta\mathrm{Tr}(\frac{1}{2} (\theta^TX^TX\theta - \theta^TX^Ty - y^TX\theta+ yy^T))
$$




Now, since $\mathrm{Tr}(A) = \mathrm{Tr}(A^T)$:

$$
\nabla_\theta J(\theta) = \frac{1}{2} \nabla_\theta(\mathrm{Tr}(\theta^TX^TX\theta) - 2 \mathrm{Tr}(\theta^TX^Ty) + \mathrm{Tr}(yy^T))
$$

Applying [16] and [17] we get:

$$
\nabla_\theta J(\theta) = \frac{1}{2} (X^TX\theta + X^TX\theta -2X^Ty )= X^TX\theta -X^Ty
$$

To minimize $J$, we just equals its gradient to 0, obtaining,

$$
X^TX\theta = X^Ty
$$

and, from here, we get a closed formula for our parameter vector $\theta$:

$$
\theta = (X^TX)^{-1}X^Ty
$$

### 9. Linear regression with Python

Suppose we have target class _y_ (say, a house's price), dependent on 3 real-valued attributes $x_1,x_2,x_3$ (for example, number of rooms, minimum tax and house age). Suppose that we have the attributes and prices for 10 houses.  _If we assume_ that the price of a house is a _linear combination_ of the three attributes,  $y = \theta_0 + \theta_1x_1 + \theta_2x_2 + \theta_3x_3$, and that we should find the $\theta$ values. Now, we can define $X$ (the design matrix) and $y$

In [44]:
X=np.array([
[1.0,1.0,1.9,1.1],
[1.0,0.1,1.0,1.0],
[1.0,1.1,1.0,0.1],
[1.0,1.1,1.1,0.9],
[1.0,2.1,1.2,1.0],
[1.0,0.9,1.0,3.0],
[1.0,3.1,0.9,3.0],
[1.0,1.9,0.9,2.1],
[1.0,0.0,-0.1,1.1],
[1.0,1.0,1.0,0.1]
])
print X


[[ 1.   1.   1.9  1.1]
 [ 1.   0.1  1.   1. ]
 [ 1.   1.1  1.   0.1]
 [ 1.   1.1  1.1  0.9]
 [ 1.   2.1  1.2  1. ]
 [ 1.   0.9  1.   3. ]
 [ 1.   3.1  0.9  3. ]
 [ 1.   1.9  0.9  2.1]
 [ 1.   0.  -0.1  1.1]
 [ 1.   1.   1.   0.1]]


In [45]:
y=np.array([5.0,2.0,4.1,3.0,5.0,1.0,2.8,2.6,-0.4,4.1])
print y


[ 5.   2.   4.1  3.   5.   1.   2.8  2.6 -0.4  4.1]


Note that we added a left column to the matrix, each cell valued with 1. This is to add a new $x_0$ attribute, as before. We can now calculate the parameters for the hyperplane that minimizes the sum of squares using the previous formula ($\theta = (X^TX)^{-1}X^Ty$):

In [54]:
theta=np.dot(np.dot(np.linalg.inv(np.dot(np.transpose(X),X)),np.transpose(X)),y)
print theta

[ 0.87782702  1.051817    2.18366554 -1.05476926]


Let's verify that our just found parameter vector, dot the attributes of each instance, gives us (approximately) the corresponding $y$ cell. We can do this for every training examples, just calculating $X\theta$

In [56]:
print np.dot(X,theta)


[ 4.91836235  2.111905    4.11301432  3.48756547  4.6522721   0.84382008
  2.93945092  2.62656285 -0.50078572  4.00783262]


Of course, a) ordinary least squares is a very simple method for regression and b) every statistical/machine learning package includes much more efficient ways of finding $\theta$. For example, using the linalg library:

In [61]:
print np.linalg.lstsq(X, y)[0]

[ 0.87782702  1.051817    2.18366554 -1.05476926]


### 10. Further Reading

The aforementioned [notes](http://cs229.stanford.edu/notes/cs229-notes1.pdf) for CS229 course by Andrew Ng are an excelente introduction to Supervised learning, and make intensive use of matrix notation. From the same course,  a [Linear Algebra Review](http://cs229.stanford.edu/section/cs229-linalg.pdf) covers mostly the same topics we have seen, with some more detail.